In [ ]:
import pandas as pd
train_final = pd.read_csv('train_final.csv')
#train_final.dropna(subset = ["sales"], inplace=True)
train_final = train_final[train_final.date > '2021-05-01']

import pandas as pd
test_data = pd.read_csv('test_data.csv')
test_data['sales']=-1

result_data = pd.concat([train_final, test_data], axis=0)
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
sku_final = sku_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
sku_final = sku_final.drop(['countryOfOrigin','productType_caption_ENG','lagerUnitType_caption'], axis=1)
sku_final = sku_final.drop(['commodity_group','productCategoryId'], axis=1)
sku_final = sku_final.drop(['productCategory_caption_ENG'], axis=1)
sku_final = sku_final.drop(['countryOfOrigin_caption'], axis=1)
sku_final = sku_final.drop(['lagerUnitTypeId','lagerUnitQuantity'], axis=1)
sku_final = sku_final.drop(['brandId','trademark'], axis=1)

result_data = pd.merge(result_data, sku_final, how="left", on=["SKU","SKU"])


#result = pd.concat([result_data, pd.get_dummies(result_data['commodity_group_caption_ENG'])], axis=1)
result = result_data.drop(['commodity_group_caption_ENG'], axis=1)
result = result.drop(['productTypeId'], axis=1)

geo_df = pd.read_csv('geo_params.csv')

sku_final = pd.read_csv('sku_final.csv')
merged_final = pd.merge(result, sku_final[['commodity_group_caption_ENG','SKU']], how="left", on=["SKU","SKU"])
result = merged_final


del merged_final
del train_final
del sku_final
del result_data
del test_data
import pandas as pd
import datetime
from catboost import CatBoostRegressor
from sklearn.feature_selection import RFECV
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
def mean_absolute_percentage_error_(y_true, y_pred): 
    return np.sum(np.abs(y_true - y_pred) / np.sum(y_true))

geoCluster_mass = result['commodity_group_caption_ENG'].unique()   
for i in range(1,len(geoCluster_mass)):
    mass = result.loc[(result['commodity_group_caption_ENG'] == geoCluster_mass[i])]
    data_dollar = pd.read_excel('drive/MyDrive/fozzy/dollar.xlsx')[1:]
    data_dollar.dropna(subset = ["Euro"], inplace=True)
    data_dollar=data_dollar.fillna(0)
    data_dollar=data_dollar.rename(columns={'Data': 'date'})
    data_dollar['date'] = data_dollar['date'].astype(str)
    outer_merged = pd.merge(mass, data_dollar, how="left", on=["date","date"])
    merged_final = outer_merged

    new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
    day_cols_dict = {}
    months = []
    for date_str in merged_final.date:
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        for index, col_name in enumerate(new_cols):
            if col_name in day_cols_dict:
                day_cols_dict[col_name].append(date.weekday() == index)
            else:
                day_cols_dict[col_name] = [date.weekday() == index]

        months.append(date.month)

    for key, value in day_cols_dict.items():
        merged_final[key] = value

    merged_final['month'] = months


    anomaly_cluster = 681
    merged_final = merged_final[merged_final.geoCluster != anomaly_cluster]
    merged_geo = pd.merge(merged_final, geo_df, how='left', on=['geoCluster', 'geoCluster'])

    merged_geo = pd.concat([merged_geo, pd.get_dummies(merged_geo['commodity_group_caption_ENG'])], axis=1)
    merged_geo = pd.concat([merged_geo, pd.get_dummies(merged_geo['geoCluster'])], axis=1)
    merged_geo = pd.concat([merged_geo, pd.get_dummies(merged_geo['month'])], axis=1)
    merged_geo = merged_geo.drop(['commodity_group_caption_ENG','geoCluster','month'], axis=1)
    print(merged_geo)


    result1 = merged_geo
    del merged_geo
    del merged_final
    del geo_df
    result1=result1.fillna(0)
    train_base = result1[result1.sales != -1]
    test_base = result1[result1.sales == -1]
    del result1
    y = train_base['sales']
    X = train_base.drop(['sales'], axis=1)
    del train_base
    model = CatBoostRegressor(iterations=250, depth=16, learning_rate=0.8,task_type="GPU", loss_function="MAE")
    model.fit(X.drop(['ID','date'], axis=1), y) 
    y_ = model.predict(X.drop(['ID','date'], axis=1))
    del X
    print(mean_absolute_percentage_error_(y, y_))
    print(i)    

    test_base_id = test_base[['ID','sales']]
    test_base_id['sales'] = model.predict(test_base.drop(['ID','date','sales'], axis=1))
    test_base_id.to_csv('out'+str(i)+'.csv',index=False)

# NEW try

In [1]:
import pandas as pd
train_final = pd.read_csv('train_final.csv')
#train_final.dropna(subset = ["sales"], inplace=True)

In [2]:
import pandas as pd
test_data = pd.read_csv('test_data.csv')
test_data['sales']=-1

In [3]:
result_data = pd.concat([train_final, test_data], axis=0)

In [4]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
sku_final = sku_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
sku_final = sku_final.drop(['countryOfOrigin','productType_caption_ENG','lagerUnitType_caption'], axis=1)
sku_final = sku_final.drop(['commodity_group','productCategoryId'], axis=1)
sku_final = sku_final.drop(['productCategory_caption_ENG'], axis=1)
sku_final = sku_final.drop(['countryOfOrigin_caption'], axis=1)
sku_final = sku_final.drop(['lagerUnitTypeId','lagerUnitQuantity'], axis=1)
sku_final = sku_final.drop(['brandId','trademark'], axis=1)

result_data = pd.merge(result_data, sku_final, how="left", on=["SKU","SKU"])

In [5]:
#result = pd.concat([result_data, pd.get_dummies(result_data['commodity_group_caption_ENG'])], axis=1)
result = result_data.drop(['commodity_group_caption_ENG'], axis=1)
result = result.drop(['productTypeId'], axis=1)

In [6]:
geo_df = pd.read_csv('geo_params.csv')

In [ ]:
import pandas as pd
import datetime
from catboost import CatBoostRegressor
from sklearn.feature_selection import RFECV
import numpy as np
import statsmodels.api as sm

def mean_absolute_percentage_error_(y_true, y_pred): 
    return np.sum(np.abs(y_true - y_pred) / np.sum(y_true))

geoCluster_mass = result['geoCluster'].unique()   
for i in range(0,len(geoCluster_mass)):
    mass = result.loc[(result_data['geoCluster'] == geoCluster_mass[i])]
    data_dollar = pd.read_excel('dollar.xlsx')[1:]
    data_dollar.dropna(subset = ["Euro"], inplace=True)
    data_dollar=data_dollar.fillna(0)
    data_dollar=data_dollar.rename(columns={'Data': 'date'})
    data_dollar['date'] = data_dollar['date'].astype(str)
    outer_merged = pd.merge(mass, data_dollar, how="left", on=["date","date"])
    merged_final = outer_merged
    new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
    
    for index, col_name in enumerate(new_cols):
        merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]
    merged_final=merged_final.fillna(0)

    anomaly_cluster = 681
    merged_final = merged_final[merged_final.geoCluster != anomaly_cluster]
    merged_geo = pd.merge(merged_final, geo_df, how='left', on=['geoCluster', 'geoCluster'])
    
    result1 = merged_geo
    train_base = result1[result1.sales != -1]
    test_base = result1[result1.sales == -1]
    
    y = train_base['sales']
    X = train_base.drop(['sales'], axis=1)
    model = CatBoostRegressor(iterations=25, depth=16, learning_rate=0.8, loss_function="RMSE",)
    model.fit(X.drop(['ID','date'], axis=1), y) 
    y_ = model.predict(X.drop(['ID','date'], axis=1))
    print(mean_absolute_percentage_error_(y, y_))
    print(i)    

    test_base_id = test_base[['ID','sales']]
    prediction = model.predict(test_base.drop(['ID','date','sales'], axis=1))
    test_base_id['sales'] = prediction
    test_data = pd.merge(test_data, test_base_id, how="left", on=["ID","ID"])

0:	learn: 0.1978983	total: 1.7ms	remaining: 40.7ms
1:	learn: 0.1560448	total: 3.17ms	remaining: 36.4ms
2:	learn: 0.1232904	total: 13.9ms	remaining: 102ms
3:	learn: 0.0985265	total: 16.2ms	remaining: 84.9ms
4:	learn: 0.0792565	total: 181ms	remaining: 722ms
5:	learn: 0.0629917	total: 186ms	remaining: 588ms
6:	learn: 0.0509075	total: 222ms	remaining: 572ms
7:	learn: 0.0375487	total: 224ms	remaining: 475ms
8:	learn: 0.0298241	total: 242ms	remaining: 430ms
9:	learn: 0.0238913	total: 252ms	remaining: 378ms
10:	learn: 0.0196979	total: 253ms	remaining: 322ms
11:	learn: 0.0157991	total: 272ms	remaining: 295ms
12:	learn: 0.0126972	total: 278ms	remaining: 256ms
13:	learn: 0.0102241	total: 287ms	remaining: 225ms
14:	learn: 0.0082627	total: 328ms	remaining: 219ms
15:	learn: 0.0067136	total: 337ms	remaining: 189ms
16:	learn: 0.0054757	total: 500ms	remaining: 235ms
17:	learn: 0.0044575	total: 823ms	remaining: 320ms
18:	learn: 0.0037086	total: 899ms	remaining: 284ms
19:	learn: 0.0028034	total: 916ms	r

<ipython-input-9-6447296013be>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_base_id['sales'] = prediction


0:	learn: 0.2789168	total: 964us	remaining: 23.1ms
1:	learn: 0.2184245	total: 25.8ms	remaining: 296ms
2:	learn: 0.1694516	total: 26.8ms	remaining: 196ms
3:	learn: 0.1419068	total: 28ms	remaining: 147ms
4:	learn: 0.1170668	total: 31.4ms	remaining: 126ms
5:	learn: 0.1005028	total: 32.6ms	remaining: 103ms
6:	learn: 0.0798867	total: 137ms	remaining: 353ms
7:	learn: 0.0639086	total: 149ms	remaining: 316ms
8:	learn: 0.0510875	total: 201ms	remaining: 358ms
9:	learn: 0.0407167	total: 226ms	remaining: 339ms
10:	learn: 0.0326019	total: 330ms	remaining: 421ms
11:	learn: 0.0260883	total: 547ms	remaining: 593ms
12:	learn: 0.0209007	total: 572ms	remaining: 528ms
13:	learn: 0.0167149	total: 575ms	remaining: 452ms
14:	learn: 0.0134051	total: 577ms	remaining: 384ms
15:	learn: 0.0107882	total: 579ms	remaining: 326ms
16:	learn: 0.0087231	total: 581ms	remaining: 273ms
17:	learn: 0.0071000	total: 583ms	remaining: 227ms
18:	learn: 0.0058328	total: 586ms	remaining: 185ms
19:	learn: 0.0048534	total: 588ms	rem

9:	learn: 0.0646236	total: 1.45s	remaining: 2.17s
10:	learn: 0.0534023	total: 1.48s	remaining: 1.88s
11:	learn: 0.0431636	total: 1.7s	remaining: 1.84s
12:	learn: 0.0346430	total: 2.15s	remaining: 1.99s
13:	learn: 0.0278264	total: 2.59s	remaining: 2.03s
14:	learn: 0.0223641	total: 3.03s	remaining: 2.02s
15:	learn: 0.0180447	total: 3.46s	remaining: 1.95s
16:	learn: 0.0145964	total: 3.9s	remaining: 1.83s
17:	learn: 0.0117162	total: 4.33s	remaining: 1.68s
18:	learn: 0.0094514	total: 4.77s	remaining: 1.51s
19:	learn: 0.0076444	total: 5.21s	remaining: 1.3s
20:	learn: 0.0062552	total: 5.65s	remaining: 1.08s
21:	learn: 0.0051602	total: 6.09s	remaining: 830ms
22:	learn: 0.0043197	total: 6.53s	remaining: 568ms
23:	learn: 0.0036734	total: 6.97s	remaining: 290ms
24:	learn: 0.0032029	total: 7.41s	remaining: 0us
0.01471110811348834
7
0:	learn: 0.0189945	total: 393us	remaining: 9.45ms
1:	learn: 0.0151875	total: 957us	remaining: 11ms
2:	learn: 0.0121491	total: 1.55ms	remaining: 11.3ms
3:	learn: 0.0097

16:	learn: 0.2601802	total: 5.35s	remaining: 2.52s
17:	learn: 0.2394940	total: 5.78s	remaining: 2.25s
18:	learn: 0.2238242	total: 6.22s	remaining: 1.96s
19:	learn: 0.2181000	total: 6.67s	remaining: 1.67s
20:	learn: 0.2155271	total: 7.11s	remaining: 1.35s
21:	learn: 0.1992109	total: 7.55s	remaining: 1.03s
22:	learn: 0.1808547	total: 7.99s	remaining: 695ms
23:	learn: 0.1677458	total: 8.43s	remaining: 351ms
24:	learn: 0.1677056	total: 8.43s	remaining: 0us
0.10557741171908883
13
0:	learn: 0.5599659	total: 15.2ms	remaining: 365ms
1:	learn: 0.4399061	total: 127ms	remaining: 1.46s
2:	learn: 0.3174759	total: 154ms	remaining: 1.13s
3:	learn: 0.2521052	total: 179ms	remaining: 942ms
4:	learn: 0.1982807	total: 181ms	remaining: 722ms
5:	learn: 0.1631928	total: 182ms	remaining: 577ms
6:	learn: 0.1335115	total: 184ms	remaining: 472ms
7:	learn: 0.1083300	total: 196ms	remaining: 417ms
8:	learn: 0.0874569	total: 255ms	remaining: 453ms
9:	learn: 0.0702523	total: 262ms	remaining: 393ms
10:	learn: 0.056439

...

24:	learn: 0.8831566	total: 1m 3s	remaining: 0us
0.4435423564102875
135
0:	learn: 0.5502491	total: 590ms	remaining: 14.2s
1:	learn: 0.4951859	total: 3.07s	remaining: 35.3s
2:	learn: 0.4768388	total: 5.58s	remaining: 41s
3:	learn: 0.4600263	total: 8.09s	remaining: 42.5s
4:	learn: 0.4515647	total: 10.6s	remaining: 42.5s
5:	learn: 0.4389009	total: 13.1s	remaining: 41.6s
6:	learn: 0.4303545	total: 15.7s	remaining: 40.3s
7:	learn: 0.4192110	total: 18.2s	remaining: 38.8s
8:	learn: 0.4107691	total: 20.8s	remaining: 37s
9:	learn: 0.4020342	total: 23.4s	remaining: 35s
10:	learn: 0.3946021	total: 25.9s	remaining: 32.9s
11:	learn: 0.3869792	total: 28.4s	remaining: 30.7s
12:	learn: 0.3814009	total: 30.9s	remaining: 28.5s
13:	learn: 0.3750449	total: 33.4s	remaining: 26.3s
14:	learn: 0.3689832	total: 36s	remaining: 24s
15:	learn: 0.3665224	total: 38.6s	remaining: 21.7s
16:	learn: 0.3621033	total: 41.2s	remaining: 19.4s
17:	learn: 0.3582471	total: 43.7s	remaining: 17s
18:	learn: 0.3537350	total: 46.3

10:	learn: 0.8512631	total: 27.8s	remaining: 35.4s
11:	learn: 0.8421686	total: 30.3s	remaining: 32.8s
12:	learn: 0.8302993	total: 32.8s	remaining: 30.3s
13:	learn: 0.8211218	total: 35.4s	remaining: 27.8s
14:	learn: 0.8127506	total: 37.9s	remaining: 25.3s
15:	learn: 0.8063133	total: 40.4s	remaining: 22.8s
16:	learn: 0.7945797	total: 43s	remaining: 20.2s
17:	learn: 0.7890827	total: 45.5s	remaining: 17.7s
18:	learn: 0.7767586	total: 48s	remaining: 15.2s
19:	learn: 0.7672649	total: 50.6s	remaining: 12.6s
20:	learn: 0.7608541	total: 53.1s	remaining: 10.1s
21:	learn: 0.7526867	total: 55.6s	remaining: 7.59s
22:	learn: 0.7468516	total: 58.2s	remaining: 5.06s
23:	learn: 0.7405412	total: 1m	remaining: 2.53s
24:	learn: 0.7311887	total: 1m 3s	remaining: 0us
0.4345540329746029
142
0:	learn: 0.6114140	total: 2.44s	remaining: 58.6s
1:	learn: 0.4860639	total: 4.87s	remaining: 56.1s
2:	learn: 0.4078013	total: 7.31s	remaining: 53.6s
3:	learn: 0.3466359	total: 9.74s	remaining: 51.1s
4:	learn: 0.2969286	t

21:	learn: 0.2304468	total: 54.3s	remaining: 7.41s
22:	learn: 0.2273994	total: 56.8s	remaining: 4.94s
23:	learn: 0.2224084	total: 59.3s	remaining: 2.47s
24:	learn: 0.2180361	total: 1m 1s	remaining: 0us
0.2606085934026137
148
0:	learn: 0.6502604	total: 606ms	remaining: 14.5s
1:	learn: 0.5793428	total: 3.1s	remaining: 35.7s
2:	learn: 0.5475926	total: 5.58s	remaining: 41s
3:	learn: 0.5263391	total: 8.06s	remaining: 42.3s
4:	learn: 0.5152759	total: 10.5s	remaining: 42.1s
5:	learn: 0.5040404	total: 13s	remaining: 41.2s
6:	learn: 0.4890165	total: 15.5s	remaining: 39.8s
7:	learn: 0.4776843	total: 18s	remaining: 38.3s
8:	learn: 0.4641003	total: 20.5s	remaining: 36.5s
9:	learn: 0.4558652	total: 23s	remaining: 34.6s
10:	learn: 0.4480324	total: 25.5s	remaining: 32.5s
11:	learn: 0.4397985	total: 28.1s	remaining: 30.4s
12:	learn: 0.4347667	total: 30.6s	remaining: 28.2s
13:	learn: 0.4262018	total: 33.1s	remaining: 26s
14:	learn: 0.4181270	total: 35.6s	remaining: 23.7s
15:	learn: 0.4134257	total: 38.

7:	learn: 0.1180988	total: 16.3s	remaining: 34.6s
8:	learn: 0.1085548	total: 18.6s	remaining: 33.1s
9:	learn: 0.0998698	total: 20.9s	remaining: 31.4s
10:	learn: 0.0995352	total: 21s	remaining: 26.7s
11:	learn: 0.0915127	total: 23.3s	remaining: 25.2s
12:	learn: 0.0854267	total: 25.6s	remaining: 23.6s
13:	learn: 0.0797470	total: 27.9s	remaining: 21.9s
14:	learn: 0.0755214	total: 30.2s	remaining: 20.2s
15:	learn: 0.0727008	total: 32.6s	remaining: 18.3s
16:	learn: 0.0698405	total: 35s	remaining: 16.4s
17:	learn: 0.0659069	total: 37.3s	remaining: 14.5s
18:	learn: 0.0619048	total: 39.6s	remaining: 12.5s
19:	learn: 0.0611998	total: 39.7s	remaining: 9.94s
20:	learn: 0.0575295	total: 42.1s	remaining: 8.01s
21:	learn: 0.0539001	total: 44.4s	remaining: 6.06s
22:	learn: 0.0521707	total: 46.7s	remaining: 4.06s
23:	learn: 0.0488010	total: 49.1s	remaining: 2.04s
24:	learn: 0.0464537	total: 51.4s	remaining: 0us
0.13432533800660454
155
0:	learn: 0.9544633	total: 619ms	remaining: 14.8s
1:	learn: 0.86340

18:	learn: 0.0284769	total: 39.9s	remaining: 12.6s
19:	learn: 0.0261404	total: 42s	remaining: 10.5s
20:	learn: 0.0252603	total: 44s	remaining: 8.39s
21:	learn: 0.0239509	total: 46.2s	remaining: 6.29s
22:	learn: 0.0227529	total: 48.3s	remaining: 4.2s
23:	learn: 0.0214846	total: 50.4s	remaining: 2.1s
24:	learn: 0.0204203	total: 52.5s	remaining: 0us
0.08364970691609348
161
0:	learn: 0.3004041	total: 1.81s	remaining: 43.5s
1:	learn: 0.2334830	total: 3.77s	remaining: 43.4s
2:	learn: 0.1799913	total: 5.76s	remaining: 42.2s
3:	learn: 0.1456800	total: 7.73s	remaining: 40.6s
4:	learn: 0.1192411	total: 9.67s	remaining: 38.7s
5:	learn: 0.0983840	total: 11.6s	remaining: 36.8s
6:	learn: 0.0826548	total: 13.6s	remaining: 34.9s
7:	learn: 0.0695651	total: 15.6s	remaining: 33.2s
8:	learn: 0.0588543	total: 17.6s	remaining: 31.2s
9:	learn: 0.0515354	total: 19.5s	remaining: 29.3s
10:	learn: 0.0451240	total: 21.4s	remaining: 27.3s
11:	learn: 0.0390955	total: 23.4s	remaining: 25.3s
12:	learn: 0.0348062	tota

In [ ]:
test_data.to_csv('out1.csv',index=False)

In [ ]:
import pandas as pd
data_dollar = pd.read_excel('dollar.xlsx')[1:]
data_dollar.dropna(subset = ["Euro"], inplace=True)
data_dollar=data_dollar.fillna(0)
data_dollar=data_dollar.rename(columns={'Data': 'date'})
data_dollar['date'] = data_dollar['date'].astype(str)
outer_merged = pd.merge(result_data, data_dollar, how="left", on=["date","date"])

In [ ]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
merged_final = pd.merge(outer_merged, sku_final, how="left", on=["SKU","SKU"])

In [ ]:
merged_final = merged_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin','productType_caption_ENG','Year','lagerUnitType_caption'], axis=1)
merged_final = merged_final.drop(['commodity_group','productCategoryId'], axis=1)
merged_final = merged_final.drop(['productCategory_caption_ENG'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin_caption'], axis=1)
merged_final = merged_final.sort_values(by=['date'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_labels = ['commodity_group_caption_ENG','geoCluster']
merged_final[cat_labels] = merged_final[cat_labels].fillna('NaN')

le = LabelEncoder()
label_encoded_df = merged_final.copy()
for col in cat_labels:
    label_encoded_df[col]=le.fit_transform(label_encoded_df[col])

In [ ]:
import datetime

merged_final = label_encoded_df
new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
for index, col_name in enumerate(new_cols):
    merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]

In [ ]:
result = pd.concat([merged_final, pd.get_dummies(merged_final['commodity_group_caption_ENG'])], axis=1)
result = result.drop(['commodity_group_caption_ENG'], axis=1)

In [44]:
test_base = result[result.sales == 0]
train_base = result[result.sales != 0]

y = train_base['sales']
X = train_base.drop(['sales'], axis=1)

# Categorical boosting

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(result, y, random_state=42)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor
from sklearn.feature_selection import RFECV
import numpy as np

def mean_absolute_percentage_error_(y_true, y_pred): 
    return np.sum(np.abs(y_true - y_pred) / np.sum(y_true))

model = CatBoostRegressor(iterations=15, depth=16, learning_rate=1, loss_function="RMSE",)
#model.fit(X.drop(['ID'], axis=1), y)

rfecv = RFECV(estimator = model,cv = 250000,scoring = 'neg_mean_absolute_percentage_error')
rfecv.fit(X.drop(['ID'], axis=1), y)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 3.1040067	total: 2.76s	remaining: 38.6s
1:	learn: 3.0472329	total: 5.2s	remaining: 33.8s
2:	learn: 3.0156002	total: 7.67s	remaining: 30.7s
3:	learn: 2.9745176	total: 10.2s	remaining: 28s
4:	learn: 2.9565490	total: 12.7s	remaining: 25.3s
5:	learn: 2.9470363	total: 15.2s	remaining: 22.8s
6:	learn: 2.8908771	total: 17.6s	remaining: 20.1s
7:	learn: 2.8775713	total: 20s	remaining: 17.5s
8:	learn: 2.8403307	total: 22.4s	remaining: 14.9s
9:	learn: 2.8223852	total: 24.9s	remaining: 12.4s
10:	learn: 2.8028115	total: 27.3s	remaining: 9.93s
11:	learn: 2.7948962	total: 29.8s	remaining: 7.45s
12:	learn: 2.7855128	total: 32.2s	remaining: 4.95s
13:	learn: 2.7707462	total: 34.8s	remaining: 2.48s
14:	learn: 2.7622066	total: 37.1s	remaining: 0us
0:	learn: 3.0966970	total: 2.57s	remaining: 36s
1:	learn: 3.0553166	total: 5.06s	remaining: 32.9s
2:	learn: 3.0207479	total: 7.48s	remaining: 29.9s
3:	learn: 2.9852801	total: 10s	remaining: 27.5s
4:	learn: 2.9530066	total: 12.7s	remaining: 25.4s
5:	lea

In [75]:
y_ = model.predict(X.drop(['ID'], axis=1))
mean_absolute_percentage_error_(y, y_)

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

# Prediction

In [68]:
y_ans = model.predict(test_base.drop(['sales','ID'], axis=1))

df_answer = pd.DataFrame(columns=['ID','sales'])
df_answer['sales'] = y_ans
df_answer['ID'] = test_base['ID'].values

for i in range(0,len(df_answer)):
    if df_answer.sales[i] < 0:
        df_answer.sales[i] = 0

df_answer

C:\Users\lysyi\AppData\Local\Temp/ipykernel_15864/957978332.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_answer.sales[i] = 0


,ID,sales
0,RR2653982,2.860516
1,RR1835556,0.640024
2,RR2653996,3.196059
3,RR2653968,2.419997
4,RR2653954,0.000000
...,...,...
1666023,RR2814995,0.000000
1666024,RR1795291,3.001508
1666025,RR2815009,3.846124
1666026,RR2815037,1.115777


In [69]:
df_answer.to_csv('out.csv',index=False)

In [9]:
import pandas as pd
data = pd.read_csv('train_final.csv')
data.dropna(subset = ["sales"], inplace=True)
data.to_csv('clean_data.csv')

In [ ]:
data.reset_index(drop=True, inplace=True)
data['date'] = data['date'].astype(str)

In [60]:
import pandas as pd
data_dollar = pd.read_excel('dollar.xlsx')[1:]
data_dollar.dropna(subset = ["Euro"], inplace=True)
data_dollar=data_dollar.fillna(0)
data_dollar=data_dollar.rename(columns={'Data': 'date'})
data_dollar['date'] = data_dollar['date'].astype(str)
outer_merged = pd.merge(data, data_dollar, how="left", on=["date","date"])
outer_merged

In [67]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
outer_merged = pd.read_csv('outer_merged.csv')
merged_final = pd.merge(outer_merged, sku_final, how="left", on=["SKU","SKU"])
merged_final.to_csv('merged_final.csv')

In [73]:
merged_final = merged_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin','productType_caption_ENG','Year','lagerUnitType_caption'], axis=1)
merged_final = merged_final.drop(['commodity_group','productCategoryId'], axis=1)
merged_final = merged_final.drop(['productCategory_caption_ENG'], axis=1)

In [100]:
merged_final = merged_final.drop(['countryOfOrigin_caption'], axis=1)

In [104]:
merged_final = merged_final.sort_values(by=['date'])

In [105]:
merged_final

,ID,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
1653540,RR52304931,2065,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,Bakery
1653541,RR52304936,2065,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,Bakery
1045940,RR43963070,2065,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,Mineral water
1045989,RR43964553,2065,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,Mineral water
1653542,RR52304937,2065,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148294,RR45619916,2482,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,Mineral water
1974295,RR55877425,2992,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,Bakery
1148107,RR45618422,2482,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,Mineral water
1148508,RR45623020,2482,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,Mineral water


In [132]:
merged_final.to_csv('merged_final_.csv')

In [57]:
import pandas as pd
merged_final = pd.read_csv('merged_final_.csv')

In [58]:
merged_final

,Unnamed: 0,ID,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,1653540,RR52304931,2065,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,Bakery
1,1653541,RR52304936,2065,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,Bakery
2,1045940,RR43963070,2065,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,Mineral water
3,1045989,RR43964553,2065,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,Mineral water
4,1653542,RR52304937,2065,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,1148294,RR45619916,2482,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,Mineral water
1981383,1974295,RR55877425,2992,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,Bakery
1981384,1148107,RR45618422,2482,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,Mineral water
1981385,1148508,RR45623020,2482,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,Mineral water


In [59]:
data1 = merged_final.drop(['ID'], axis=1)

In [60]:
data1

,Unnamed: 0,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,1653540,2065,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,Bakery
1,1653541,2065,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,Bakery
2,1045940,2065,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,Mineral water
3,1045989,2065,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,Mineral water
4,1653542,2065,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,1148294,2482,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,Mineral water
1981383,1974295,2992,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,Bakery
1981384,1148107,2482,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,Mineral water
1981385,1148508,2482,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,Mineral water


In [61]:
y = data1['sales']
X = data1.drop(['sales'], axis=1)

In [118]:
#from statsmodels.tsa.seasonal import seasonal_decompose
#season = seasonal_decompose(clean_data['y_total_trans'][0:106].values, model='multiplicative', period=53)
#seson_var = (season.seasonal - 1)[0:53].tolist()
#clean_data['season'] = (seson_var+seson_var+seson_var+seson_var)[0:len(clean_data['y_total_trans'])]
#print(clean_data)

In [63]:
from sklearn.preprocessing import LabelEncoder

cat_labels = ['commodity_group_caption_ENG','geoCluster']
data1[cat_labels] = data1[cat_labels].fillna('NaN')

le = LabelEncoder()
label_encoded_df = data1.copy()
for col in cat_labels:
    label_encoded_df[col]=le.fit_transform(label_encoded_df[col])


In [64]:
label_encoded_df

,Unnamed: 0,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,1653540,218,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,0
1,1653541,218,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,0
2,1045940,218,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,2
3,1045989,218,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,2
4,1653542,218,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,1148294,400,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,2
1981383,1974295,481,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,0
1981384,1148107,400,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,2
1981385,1148508,400,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,2


In [65]:
label_encoded_df = label_encoded_df.drop(['Unnamed: 0'], axis=1)
#label_encoded_df = label_encoded_df.drop(['date'], axis=1)

In [66]:
label_encoded_df

,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,218,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,0
1,218,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,0
2,218,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,2
3,218,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,2
4,218,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,400,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,2
1981383,481,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,0
1981384,400,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,2
1981385,400,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,2


In [68]:
import datetime

merged_final = label_encoded_df
new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
for index, col_name in enumerate(new_cols):
    merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]

In [69]:
merged_final

,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,...,lagerUnitTypeId,trademark,commodity_group_caption_ENG,is_monday,is_tuesday,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday
0,218,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,...,1,NaN,0,False,False,False,False,True,False,False
1,218,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,...,1,NaN,0,False,False,True,False,False,False,False
2,218,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,...,3,1323.0,2,False,False,True,False,False,False,False
3,218,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,...,3,1323.0,2,False,False,True,False,False,False,False
4,218,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,...,1,NaN,0,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,400,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,...,3,1839.0,2,True,False,False,False,False,False,False
1981383,481,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,...,1,4043.0,0,True,False,False,False,False,False,False
1981384,400,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,...,3,2781.0,2,True,False,False,False,False,False,False
1981385,400,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,...,3,2781.0,2,True,False,False,False,False,False,False


In [89]:
result = pd.concat([merged_final, pd.get_dummies(merged_final['commodity_group_caption_ENG'])], axis=1)

In [91]:
result = result.drop(['commodity_group_caption_ENG'], axis=1)

In [98]:
result1 = pd.concat([result, pd.get_dummies(merged_final['geoCluster'])], axis=1)
result1 = result1.drop(['geoCluster'], axis=1)

In [80]:
#merged_final = merged_final.drop(['date'], axis=1)
#merged_final = merged_final.fillna(0)
#merged_final = merged_final.drop(['sales'], axis=1)

In [99]:
result1 = result1.fillna(0)

In [165]:
#import statsmodels.api as sm
#est = sm.OLS(y.astype(float), sm.add_constant(result1.astype(float)))
#est2 = est.fit()

#print(est2.summary())
#print('\n\n')
#print(est2.params)

In [ ]:
y_ = est2.predict(sm.add_constant(result1.astype(float)))

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, y_)

# Cat

In [171]:
X_train

,geoCluster,SKU,price,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,...,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday,0,1,2,3,4
36988,152,32490,21.39,1.0,26.9714,29.2424,0,4754.0,0.0,1.0,...,False,False,True,False,False,0,0,0,1,0
1947807,224,828126,71.09,0.0,27.3047,32.2482,0,383.0,0.0,1.0,...,False,True,False,False,False,1,0,0,0,0
1911446,510,538208,96.59,0.0,27.2862,32.3191,0,4755.0,0.0,1.0,...,False,False,False,True,False,0,0,0,1,0
1972933,436,571805,339.49,0.0,27.2905,32.2369,0,1897.0,7420.0,1.0,...,False,False,False,False,True,0,1,0,0,0
914629,169,775007,17.59,0.0,27.8461,33.6478,0,400.0,7418.0,115.0,...,False,False,False,False,True,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,202,810552,241.39,0.0,28.3762,33.4343,0,1894.0,2737.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1414414,400,437495,22.79,1.0,27.7715,33.5577,0,1014.0,8314.0,50.0,...,True,False,False,False,False,1,0,0,0,0
131932,371,596031,36.99,0.0,27.3606,31.2554,0,4698.0,0.0,400.0,...,False,False,False,False,False,1,0,0,0,0
671155,328,612845,49.79,1.0,28.0609,33.9888,0,3282.0,967.0,305.0,...,False,False,False,True,False,0,0,0,0,1


In [122]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(result, y, random_state=42)

from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=150, depth=15, learning_rate=0.5)
model.fit(X_train, y_train)
y_ = model.predict(X_test)

from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_error(y_test, y_)

0:	learn: 3.2653354	total: 1.5s	remaining: 3m 43s
1:	learn: 3.1478214	total: 2.99s	remaining: 3m 41s
2:	learn: 3.0924155	total: 4.37s	remaining: 3m 34s
3:	learn: 3.0674175	total: 5.81s	remaining: 3m 32s
4:	learn: 3.0450895	total: 7.19s	remaining: 3m 28s
5:	learn: 3.0271329	total: 8.56s	remaining: 3m 25s
6:	learn: 3.0119238	total: 9.97s	remaining: 3m 23s
7:	learn: 2.9837981	total: 11.4s	remaining: 3m 21s
8:	learn: 2.9552405	total: 12.8s	remaining: 3m 19s
9:	learn: 2.9416542	total: 14.2s	remaining: 3m 18s
10:	learn: 2.9343839	total: 15.5s	remaining: 3m 16s
11:	learn: 2.9221164	total: 16.9s	remaining: 3m 14s
12:	learn: 2.8996250	total: 18.2s	remaining: 3m 12s
13:	learn: 2.8944248	total: 19.6s	remaining: 3m 10s
14:	learn: 2.8879510	total: 20.9s	remaining: 3m 8s
15:	learn: 2.8782918	total: 22.3s	remaining: 3m 7s
16:	learn: 2.8741947	total: 23.7s	remaining: 3m 5s
17:	learn: 2.8529538	total: 25.1s	remaining: 3m 4s
18:	learn: 2.8325689	total: 26.5s	remaining: 3m 2s
19:	learn: 2.8278115	total: 

1.4295802185238677

In [136]:
import numpy as np
def mean_absolute_percentage_error_(y_true, y_pred): 
    return np.sum(np.abs(y_true - y_pred) / np.sum(y_true))

In [137]:
mean_absolute_percentage_error_(y_test, y_)

0.4764156681510358

In [226]:
X_train

,geoCluster,SKU,price,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,...,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday,0,1,2,3,4
36988,152,32490,21.39,1.0,26.9714,29.2424,0,4754.0,0.0,1.0,...,False,False,True,False,False,0,0,0,1,0
1947807,224,828126,71.09,0.0,27.3047,32.2482,0,383.0,0.0,1.0,...,False,True,False,False,False,1,0,0,0,0
1911446,510,538208,96.59,0.0,27.2862,32.3191,0,4755.0,0.0,1.0,...,False,False,False,True,False,0,0,0,1,0
1972933,436,571805,339.49,0.0,27.2905,32.2369,0,1897.0,7420.0,1.0,...,False,False,False,False,True,0,1,0,0,0
914629,169,775007,17.59,0.0,27.8461,33.6478,0,400.0,7418.0,115.0,...,False,False,False,False,True,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,202,810552,241.39,0.0,28.3762,33.4343,0,1894.0,2737.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1414414,400,437495,22.79,1.0,27.7715,33.5577,0,1014.0,8314.0,50.0,...,True,False,False,False,False,1,0,0,0,0
131932,371,596031,36.99,0.0,27.3606,31.2554,0,4698.0,0.0,400.0,...,False,False,False,False,False,1,0,0,0,0
671155,328,612845,49.79,1.0,28.0609,33.9888,0,3282.0,967.0,305.0,...,False,False,False,True,False,0,0,0,0,1


# Predict

In [234]:
import pandas as pd
test_data = pd.read_csv('test_data.csv')

In [235]:
idd = test_data['ID']
test_data = test_data.drop(['ID'], axis=1)
#test_data = test_data.drop(['geoCluster'], axis=1)

In [236]:
import pandas as pd
data_dollar = pd.read_excel('dollar.xlsx')[1:]
data_dollar.dropna(subset = ["Euro"], inplace=True)
data_dollar=data_dollar.fillna(0)
data_dollar=data_dollar.rename(columns={'Data': 'date'})
data_dollar['date'] = data_dollar['date'].astype(str)
outer_merged = pd.merge(test_data, data_dollar, how="left", on=["date","date"])

In [237]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
merged_final = pd.merge(outer_merged, sku_final, how="left", on=["SKU","SKU"])

In [238]:
merged_final = merged_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin','productType_caption_ENG','Year','lagerUnitType_caption'], axis=1)
merged_final = merged_final.drop(['commodity_group','productCategoryId'], axis=1)
merged_final = merged_final.drop(['productCategory_caption_ENG'], axis=1)

In [239]:
merged_final = merged_final.drop(['countryOfOrigin_caption'], axis=1)

In [240]:
merged_final = merged_final.sort_values(by=['date'])

In [241]:
import datetime

new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
for index, col_name in enumerate(new_cols):
    merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]

In [242]:
result_ = pd.concat([merged_final, pd.get_dummies(merged_final['commodity_group_caption_ENG'])], axis=1)
result_ = result_.drop(['commodity_group_caption_ENG','date'], axis=1)

In [243]:
result_

,geoCluster,SKU,price,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,...,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday,Bakery,Cheese,Mineral water,Tropical fruits,Yogurts
0,21,32485,66.69,0.0,27.2132,32.0313,0,4752.0,NaN,1.0,...,False,False,False,False,False,0,0,0,1,0
1651034,3168,775006,36.69,0.0,27.2132,32.0313,0,400.0,7418.0,115.0,...,False,False,False,False,False,0,0,0,0,1
209888,2019,719934,268.49,0.0,27.2132,32.0313,0,1895.0,2089.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1026746,2246,49215,283.99,0.0,27.2132,32.0313,0,4758.0,NaN,1.0,...,False,False,False,False,False,0,0,0,1,0
1026732,2246,49207,283.99,0.0,27.2132,32.0313,0,4768.0,NaN,1.0,...,False,False,False,False,False,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036713,2248,566113,9.99,0.0,26.8168,31.8946,0,3104.0,1594.0,36.0,...,False,False,False,False,False,0,0,0,0,1
1036727,2248,566929,9.99,0.0,26.8168,31.8946,0,1897.0,6666.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1036741,2248,569082,9.99,0.0,26.8168,31.8946,0,3283.0,967.0,300.0,...,False,False,False,False,False,0,0,0,0,1
1036587,2248,555470,18.49,0.0,26.8168,31.8946,0,3104.0,1594.0,36.0,...,False,False,False,False,False,0,0,0,0,1


In [203]:
y_ans = model.predict(result_)

df_answer = pd.DataFrame(columns=['ID','sales'])
df_answer['sales'] = y_ans
df_answer['ID'] = idd

df_answer

df_answer.to_csv('out.csv',index=False)

In [183]:
df_answer.to_csv('out.csv',index=False)

In [44]:
import pandas as pd
train_final = pd.read_csv('train_final.csv')

,ID,geoCluster,SKU,date,price,sales
0,RR27956447,21,32485,2021-07-08,39.69,0.9
1,RR27956448,21,32485,2021-07-09,NaN,NaN
2,RR27956449,21,32485,2021-07-10,NaN,NaN
3,RR27956450,21,32485,2021-07-11,NaN,NaN
4,RR27956451,21,32485,2021-07-12,NaN,NaN
...,...,...,...,...,...,...
27956440,RR55912887,3230,841486,2021-07-15,NaN,NaN
27956441,RR55912888,3230,841486,2021-07-16,NaN,NaN
27956442,RR55912889,3230,841486,2021-07-17,NaN,NaN
27956443,RR55912890,3230,841486,2021-07-18,NaN,NaN


In [51]:
train_final[train_final.date > '2021-07-08']

,ID,geoCluster,SKU,date,price,sales
1,RR27956448,21,32485,2021-07-09,NaN,NaN
2,RR27956449,21,32485,2021-07-10,NaN,NaN
3,RR27956450,21,32485,2021-07-11,NaN,NaN
4,RR27956451,21,32485,2021-07-12,NaN,NaN
5,RR27956452,21,32485,2021-07-13,NaN,NaN
...,...,...,...,...,...,...
27956440,RR55912887,3230,841486,2021-07-15,NaN,NaN
27956441,RR55912888,3230,841486,2021-07-16,NaN,NaN
27956442,RR55912889,3230,841486,2021-07-17,NaN,NaN
27956443,RR55912890,3230,841486,2021-07-18,NaN,NaN


In [117]:
import pandas as pd
test_data = pd.read_csv('test_data.csv')
test_data = test_data['ID']

print(len(test_data))

out = pd.read_csv('out0.csv')
for j in range(1,20):
    try:
        out = pd.concat([out, pd.read_csv('out'+str(j)+'.csv')], axis=0)
        print(j)
    except: 
        pass
    
print(out)
dd = pd.merge(test_data, out, how="left", on=["ID","ID"])

1666028
1
2
3
4
               ID         sales
0       RR1666030 -2.117582e-22
1       RR1666031 -2.117582e-22
2       RR1666032 -1.965095e-14
3       RR1666033 -2.117582e-22
4       RR1666034 -2.117582e-22
...           ...           ...
355973  RR3331591  0.000000e+00
355974  RR3331592  0.000000e+00
355975  RR3331593  0.000000e+00
355976  RR3331594  0.000000e+00
355977  RR3331595  0.000000e+00

[1666014 rows x 2 columns]


In [118]:
dd['sales'] = dd.sales.apply(func=lambda x: x if x >= 0 else 0)
dd.fillna(0).to_csv('out_final.csv',index=False)

In [119]:
dd

,ID,sales
0,RR1666030,0.0
1,RR1666031,0.0
2,RR1666032,0.0
3,RR1666033,0.0
4,RR1666034,0.0
...,...,...
1666023,RR3332053,0.0
1666024,RR3332054,0.0
1666025,RR3332055,0.0
1666026,RR3332056,0.8


In [34]:
import pandas as pd
test_data = pd.read_csv('test_data.csv')
test_data = test_data['ID']

In [35]:
print(len(test_data))

1666028


In [36]:
out = pd.read_csv('ans/ans/out0.csv')
for j in range(1,3000):
    try:
        out = pd.concat([out, pd.read_csv('ans/ans/out'+str(j)+'.csv')], axis=0)
        print(j)
    except: 
        pass
    
print(out)
#test_data = pd.merge(test_data, out, how="left", on=["ID","ID"])

1
2
3
4
5
6
7
9
10
12
13
14
15
16
17
18
19
21
22
23
26
28
30
32
37
38
41
42
43
46
47
48
50
52
54
57
62
63
65
66
67
69
70
78
80
82
85
86
87
89
94
104
105
116
118
119
120
126
129
130
139
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
270
271
272
274
275
276
277
278
279
280
281
282
283
284
286
287
288
289
290
292
293
294
295
296
301
302
304
305
306
307
308
309
310
311
312
316
317
318
319
321
322
323
324
326
327
328
329
330
331
332
333
334
338
339
340
341
342
343
345
346
347
348
350
351
353
357
358
360
361
362
363
364
365
366
367
368
369
370
371
373
374
375

In [5]:
dd = pd.DataFrame(test_data)
dd['sales'] = dd.sales.apply(func=lambda x: x if x >= 0 else 0)
dd

In [37]:
out = pd.DataFrame(out)
out['sales'] = out.sales.apply(func=lambda x: x if x >= 0 else 0)
out

,ID,sales
0,RR1666030,3.200001
1,RR1666031,2.700000
2,RR1666032,0.000000
3,RR1666033,0.000000
4,RR1666034,2.000004
...,...,...
9,RR3330107,0.000000
10,RR3330108,0.000000
11,RR3330109,0.000000
12,RR3330110,0.000000


In [38]:
out.fillna(0).to_csv('out_by_product.csv',index=False)

In [61]:
import pandas as pd
out = pd.read_csv('out_main.csv')
dd = pd.read_csv('out_by_product.csv')

In [62]:
out

,ID,sales
0,RR1666030,0.0
1,RR1666031,0.0
2,RR1666032,0.0
3,RR1666033,0.0
4,RR1666034,0.0
...,...,...
1666023,RR3332053,0.0
1666024,RR3332054,0.0
1666025,RR3332055,0.0
1666026,RR3332056,0.0


In [63]:
dd

,ID,sales
0,RR1666030,3.200001
1,RR1666031,2.700000
2,RR1666032,0.000000
3,RR1666033,0.000000
4,RR1666034,2.000004
...,...,...
1449611,RR3330107,0.000000
1449612,RR3330108,0.000000
1449613,RR3330109,0.000000
1449614,RR3330110,0.000000


In [64]:
diff = pd.concat([dd['ID'],out['ID']]).drop_duplicates(keep=False)

In [65]:
diff.reset_index(drop=True, inplace=True)

In [66]:
diff

0         RR1667416
1         RR1667417
2         RR1667418
3         RR1667419
4         RR1667420
            ...    
216407    RR3331731
216408    RR3331732
216409    RR3331733
216410    RR3331734
216411    RR3331735
Name: ID, Length: 216412, dtype: object

In [75]:
result1 = pd.concat([dd, pd.merge(diff, dd, how="left", on=["ID","ID"])], axis=0)

In [79]:
result1.reset_index(drop=True, inplace=True)

In [77]:
#result1 = pd.DataFrame(test_data)
result1['sales'] = result1.sales.apply(func=lambda x: x if x >= 0 else 0)

In [80]:
result1

,ID,sales
0,RR1666030,3.200001
1,RR1666031,2.700000
2,RR1666032,0.000000
3,RR1666033,0.000000
4,RR1666034,2.000004
...,...,...
1666023,RR3331731,0.000000
1666024,RR3331732,0.000000
1666025,RR3331733,0.000000
1666026,RR3331734,0.000000


In [81]:
result1.fillna(0).to_csv('out_by_product_result.csv',index=False)